# 01_prep_insitu

This notebook harmonizes insitu temperature data from water stations to a common format. The output files in /data/parsed are expected to have the following columns:

- **dt_utc**:     datetime string (UTC) formatted in "YYYY-MM-dd HH:mm:ss"
- **station_id**:   station identifier
- **station_name**: station name
- **spm**:  insitu suspended sediment concentration value
- **river**:        associted river name
- **geometry**:     wgs84 point coordinate of sampling position
- **source**:       dataset source

In [ ]:
import hvplot.pandas
import hvplot.xarray
import xarray as xr
import geopandas as gpd
import pandas as pd
import numpy as np
from pathlib import Path
import shapely

In [ ]:
selector = ['dt_utc', 'station_id', 'station_name', 'spm', 'river', 'geometry', 'source', 'type']

### WMQN

In [ ]:
paths_data = list(Path(f'../mrc_webscrapper/outputs/csv/Total Suspended Solids').rglob(f'*.csv'))

df_data = pd.DataFrame([])
for path in paths_data:
    df_temp = pd.read_csv(path, dtype={'station_code':'str'})
    df_temp['dt_utc'] = pd.to_datetime(df_temp['date']).dt.tz_convert('UTC').dt.tz_localize(None)
    #df_temp['med_frq'] = np.median(np.diff(df_temp.dt64))
    df_data = pd.concat([df_data, df_temp])

df_data_wqmn = df_data.copy()
df_data_wqmn['source'] = 'WQMN'
df_data_wqmn['type'] = 'near-surface'
df_data_wqmn = df_data_wqmn.rename(columns={'value': 'spm', 'station_code': 'station_id'})
gdf_data_wqmn = gpd.GeoDataFrame(df_data_wqmn, crs='epsg:4326',
                                 geometry=df_data_wqmn.apply(lambda row: shapely.geometry.Point((row.lon, row.lat)), axis=1)).reset_index().drop(columns=['index', 'lat', 'lon'])
gdf_data_wqmn = gdf_data_wqmn.reindex(columns=selector)

### Hydromet-Mekong & DSMP

In [ ]:
paths_data = list(Path(f'../mrc_webscrapper/outputs/csv/Sediment Concentration').rglob(f'*.csv'))

df_data = pd.DataFrame([])
for path in paths_data:
    df_temp = pd.read_csv(path, dtype={'station_code':'str'})
    df_temp['dt_utc'] = pd.to_datetime(df_temp['date']).dt.tz_convert('UTC').dt.tz_localize(None)
    #df_temp['med_frq'] = np.median(np.diff(df_temp.dt64))
    df_data = pd.concat([df_data, df_temp])

df_data['type'] = 'depth-integrated'
df_data = df_data.rename(columns={'value': 'spm', 'station_code': 'station_id'})

df_data_dsmp = df_data.copy().loc[df_data.identifier.str.contains('Measured DSMP')]
df_data_dsmp['source'] = 'DSMP'
gdf_data_dsmp = gpd.GeoDataFrame(df_data_dsmp, crs='epsg:4326',
                                 geometry=df_data_dsmp.apply(lambda row: shapely.geometry.Point((row.lon, row.lat)), axis=1)).reset_index().drop(columns=['index', 'lat', 'lon', 'identifier'])
gdf_data_dsmp = gdf_data_dsmp.reindex(columns=selector)

df_data_hydrom = df_data.copy().loc[~df_data.identifier.str.contains('Measured DSMP')]
df_data_hydrom['source'] = 'Hydromet'
gdf_data_hydrom = gpd.GeoDataFrame(df_data_hydrom, crs='epsg:4326',
                                 geometry=df_data_hydrom.apply(lambda row: shapely.geometry.Point((row.lon, row.lat)), axis=1)).reset_index().drop(columns=['index', 'lat', 'lon', 'identifier'])
gdf_data_hydrom = gdf_data_hydrom.reindex(columns=selector)

### SIWRP data

In [ ]:
path_data = Path(r"E:\git\SOS-Water\WP3\T3.2\notebooks\tsm_mapper\data\siwrp\siwrp_data.csv")

df_data = pd.read_csv(path_data)
df_data['dt_utc'] = pd.to_datetime(df_data['dt_loc_estimated']).dt.tz_convert('UTC').dt.tz_localize(None)
df_data['type'] = 'depth-integrated'
df_data['station_name'] = df_data.station_id
df_data = df_data.rename(columns={'value': 'spm'})
df_data = df_data.drop(columns=['parameter_id', 'date'])
df_data['source'] = 'SIWRP'

gdf_data_siwrp = gpd.GeoDataFrame(df_data, crs='epsg:4326',
                                 geometry=df_data.apply(lambda row: shapely.geometry.Point((row.longitude, row.latitude)), axis=1)).reset_index().drop(columns=['index', 'latitude', 'longitude'])
gdf_data_siwrp = gdf_data_siwrp.reindex(columns=selector)

### Map

In [ ]:
gdf_data_dsmp.groupby('station_id').first().hvplot(geo=True, width=500, tiles='CartoLight', label='DSMP', alpha=0.75) * \
gdf_data_wqmn.groupby('station_id').first().hvplot(geo=True, tiles='CartoLight', label='WQMN', alpha=0.75) * \
gdf_data_hydrom.groupby('station_id').first().hvplot(geo=True, tiles='CartoLight', label='Hydromet', alpha=0.75) * \
gdf_data_siwrp.groupby('station_id').first().hvplot(geo=True, tiles='CartoLight', label='SIWRP', alpha=0.75)

# Export data

In [ ]:
from shapely.geometry import Point
# merge datasets
gdf_data = pd.concat([gdf_data_dsmp, gdf_data_wqmn, gdf_data_hydrom, gdf_data_siwrp])

# apply manual coordinate corrections
path_corr = Path(r"E:\git\SOS-Water\WP3\T3.2\notebooks\tsm_mapper\data\coordinate_corrections.csv")
df_corr = pd.read_csv(path_corr)
gdf_corr = gpd.GeoDataFrame(
    df_corr,
    geometry=[Point(xy) for xy in zip(df_corr.lon_corr, df_corr.lat_corr)],
    crs="EPSG:4326"
)
dict_corr = gdf_corr.set_index('station_id')['geometry'].to_dict()
gdf_data_corr = gdf_data.copy()
gdf_data_corr['geometry_og'] = gdf_data_corr['geometry']
gdf_data_corr['geometry'] = gdf_data_corr['station_id'].map(dict_corr)
gdf_data_corr['geometry'] = gdf_data_corr.apply(lambda row: row['geometry_og'] if not(row['geometry']) else row['geometry'], axis=1)
gdf_data_corr = gdf_data_corr.drop(columns=['geometry_og'])
gdf_data = gdf_data_corr



In [ ]:
gdf_data.to_csv('input/insitu_data.csv')